In [ ]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
model = pybamm.lithium_ion.SPM()
experiment = pybamm.Experiment(["Discharge at 1C for 1 hour"])

sim = pybamm.Simulation(model, experiment=experiment)
solution = sim.solve()

time = solution.t
voltage = solution["Terminal voltage [V]"](time)

In [ ]:
c_avg = solution[
    "X-averaged negative particle concentration [mol.m-3]"
].entries.mean(axis=0)
c_avg_norm = c_avg / c_avg[0]  # Normalizing concentration

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(time, voltage, color="black", lw=2)
v_marker = ax1.plot([time[0]], [voltage[0]], "ro")[0]

ax1.set_xlabel("Time (s)")
ax1.set_ylabel("Voltage (V)")
ax1.set_title("Terminal Voltage")
ax1.grid(True)

ax2.plot(time, c_avg_norm, lw=2)
c_marker = ax2.plot([time[0]], [c_avg_norm[0]], "ro")[0]

ax2.set_xlabel("Time (s)")
ax2.set_ylabel("Normalized Avg. Concentration")
ax2.set_title("Negative Particle (Averaged)")
ax2.set_ylim(0.9, 1.01)
ax2.grid(True)

suptitle = fig.suptitle("", fontsize=14)

def update(frame):
    t = time[frame]

    v_marker.set_data([t], [voltage[frame]])
    c_marker.set_data([t], [c_avg_norm[frame]])

    suptitle.set_text(f"SPM – 1C Discharge | Time = {int(t)} s")
    return v_marker, c_marker, suptitle

ani = FuncAnimation(
    fig,
    update,
    frames=len(time),
    interval=50,
    blit=False
)

HTML(ani.to_jshtml())
